# Прием и обработка твитов.
# Начальная проба пера
# Окончательная версия со скользящим окном в файле apply_model_window

## Инициализация

In [1]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.toree.kernel.api
import java.util.Calendar

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [3]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_6cb85880407d


pipeline_6cb85880407d

## Определяем схему и инициируем потоковый датасет

In [4]:
val inputStreamPath = "/home/jovyan/work/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("timestamp", TimestampType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/work/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(timestamp,TimestampType,true))
inputDF = [tweet: string, hiddentargetclue: int ... 1 more field]


[tweet: string, hiddentargetclue: int ... 1 more field]

## Глобальная переменная для удобства просмотра датасета в отдельной ячейке

In [5]:
var globDF:DataFrame = null

globDF: org.apache.spark.sql.DataFrame = null


## Внимание!
- К сожалению, я не нашел внятной доки по апи ядра toree, чтобы нормально выводить оперативно изменяющийся датасет в этом ноутбуке
- В связи с этим см. след. пункт
- В блоке приема твитов ниже вывод датасета для удобства просмотра осуществляется через глобальную переменную globDF, которая просматривается при помощи выполнения блока, следующего за приемом твитов

## Микробатч приема твитов

In [6]:
// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

var fRuns = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            print(s"${Calendar.getInstance().toInstant} - loaded from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
            globDF = 
                model.transform(batchDF)
                    .select(
                        $"timestamp",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
        } catch {
            case e:Throwable => {
                print(e.getMessage.replaceAll("\n"," "))
                print(13.toChar)
            }
        }
    }
}.start()

getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))
fRuns = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e24fb48


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6e24fb48

## Блок просмотра результата
- Этот блок предназначен для просмотра результата потоковой обработки твитов и применения модели из предыдущего блока
- Каждый раз при выполнении этого блока будет выводиться оперативное состояние датасета из блока выше

In [9]:
%%dataframe --limit=100
globDF

timestamp,tweet,Negative Probability
2020-01-29 17:31:23.655,@insertcotku I have no clothes,0.5302611691895585
2020-01-29 17:31:23.655,@nick_crazys is it so wrong??..,0.5032726251067058
2020-01-29 17:31:23.655,"thinking my cold turned into a sinus infection, my teeth are hurting and I am so exhausted. I can feel it all thru my sinuses",0.5760119138808307
2020-01-29 17:31:23.655,Can't find a literary agent for my book,0.5673572374378179
2020-01-29 17:31:23.655,herman has given up on my late night antics and is now sleeping with my early to bed roomy. at least I have kitty-fat mischa to cuddle.,0.5426540290651356
2020-01-29 17:31:23.655,poopy we arent going to the 4 floors mall,0.4884236449783609
2020-01-29 17:31:23.655,bleh... wishin' it was the weekend,0.4884236449783609
2020-01-29 17:31:23.655,@RayOnativia Oh I know! I love basically anything and everything. I never really have any favorite bands because my mood is what bases it,0.5063001492163618
2020-01-29 17:31:23.655,Wow what a fun night! Epic birthday party! Thanks to everyone who came... Hope u had as much fun as i did haha,0.43169833817072273
2020-01-29 17:31:23.655,@jacqueline47 as long as you're not going to melt LOL. Hubby has an umbrella set up to keep him dry when he grills,0.4743131203329651


## Останов чтения потока

In [10]:
globDF = null
stream.stop()

globDF: org.apache.spark.sql.DataFrame = null
